In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
def download_data(symbol="BTC-USD", start="2022-01-01", end=None):
    if end is None:
        end = datetime.now().strftime('%Y-%m-%d')
    
    data = yf.download(symbol, start=start, end=end)
    data.reset_index(inplace=True)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    return data

In [3]:
# Temel göstergelerin hesaplanması
def calculate_indicators(data):
    # SMA hesaplama
    data['SMA20'] = data['Close'].rolling(window=20).mean()
    data['SMA50'] = data['Close'].rolling(window=50).mean()
    
    # RSI hesaplama
    delta = data['Close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    
    # MACD hesaplama
    exp1 = data['Close'].ewm(span=12, adjust=False).mean()
    exp2 = data['Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = exp1 - exp2
    data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()
    data['MACD_Histogram'] = data['MACD'] - data['Signal_Line']
    
    return data


In [4]:
# ADX hesaplama
def calculate_adx(data, period=14):
    # True Range hesaplama
    data['TR'] = pd.DataFrame([
        data['High'] - data['Low'],
        abs(data['High'] - data['Close'].shift(1)),
        abs(data['Low'] - data['Close'].shift(1))
    ]).max()
    
    # +DM ve -DM hesaplama
    data['+DM'] = (data['High'] - data['High'].shift(1)).clip(lower=0)
    data['-DM'] = (data['Low'].shift(1) - data['Low']).clip(lower=0)
    
    # TR, +DM ve -DM için smoothing
    TR_smooth = data['TR'].rolling(window=period).mean()
    plus_DM_smooth = data['+DM'].rolling(window=period).mean()
    minus_DM_smooth = data['-DM'].rolling(window=period).mean()
    
    # +DI ve -DI hesaplama
    data['+DI'] = 100 * (plus_DM_smooth / TR_smooth)
    data['-DI'] = 100 * (minus_DM_smooth / TR_smooth)
    
    # ADX hesaplama
    DX = 100 * abs((data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI']))
    data['ADX'] = DX.rolling(window=period).mean()
    
    return data

In [5]:
# Volatilite filtresi
def add_volatility_filter(data, window=20, std_dev=2):
    data['BB_middle'] = data['Close'].rolling(window=window).mean()
    bb_std = data['Close'].rolling(window=window).std()
    data['BB_upper'] = data['BB_middle'] + (bb_std * std_dev)
    data['BB_lower'] = data['BB_middle'] - (bb_std * std_dev)
    
    # Volatilite filtresi
    data['Volatility_Signal'] = 1
    data.loc[data['Close'] > data['BB_upper'], 'Volatility_Signal'] = 0
    data.loc[data['Close'] < data['BB_lower'], 'Volatility_Signal'] = 0
    
    return data

In [6]:
# Sinyal oluşturma fonksiyonunu güncelleyelim
def generate_signals(data):
    # Temel sinyaller
    data['SMA_Signal'] = 0
    data.loc[data['SMA20'] > data['SMA50'], 'SMA_Signal'] = 1
    data.loc[data['SMA20'] < data['SMA50'], 'SMA_Signal'] = -1
    
    data['RSI_Signal'] = 0
    data.loc[data['RSI'] < 35, 'RSI_Signal'] = 1  # RSI eşiklerini genişlettik
    data.loc[data['RSI'] > 65, 'RSI_Signal'] = -1
    
    data['MACD_Signal'] = 0
    data.loc[data['MACD'] > data['Signal_Line'], 'MACD_Signal'] = 1
    data.loc[data['MACD'] < data['Signal_Line'], 'MACD_Signal'] = -1
    
    # Trend gücü filtresi - ADX eşiğini düşürdük
    data['Strong_Trend'] = data['ADX'] > 20
    
    # Kombine sinyal - koşulları yumuşattık
    data['Combined_Signal'] = 0
    
    # Alış koşulları
    buy_condition = (
        (data['SMA_Signal'] + data['RSI_Signal'] + data['MACD_Signal'] >= 1) &  # En az 1 pozitif sinyal
        data['Strong_Trend'] &  # Trend güçlü olmalı
        (data['Close'] > data['SMA20'])  # Fiyat 20 günlük ortalamanın üstünde
    )
    
    # Satış koşulları
    sell_condition = (
        (data['SMA_Signal'] + data['RSI_Signal'] + data['MACD_Signal'] <= -1) &  # En az 1 negatif sinyal
        (data['Close'] < data['SMA20'])  # Fiyat 20 günlük ortalamanın altında
    )
    
    data.loc[buy_condition, 'Combined_Signal'] = 1
    data.loc[sell_condition, 'Combined_Signal'] = -1
    
    return data

In [7]:
# Risk yönetimi fonksiyonunu güncelleyelim
def apply_risk_management(data, stop_loss_pct=0.03, take_profit_pct=0.06):  # Risk parametrelerini güncelledik
    data['Position'] = 0
    position_open = False
    entry_price = 0
    trailing_stop = 0
    
    for i in range(1, len(data)):
        current_price = data['Close'].iloc[i]
        
        if not position_open and data['Combined_Signal'].iloc[i] == 1:
            # Yeni pozisyon aç
            data.loc[data.index[i], 'Position'] = 1
            position_open = True
            entry_price = current_price
            trailing_stop = entry_price * (1 - stop_loss_pct)
            
        elif position_open:
            # Trailing stop güncelleme
            if current_price > entry_price:
                new_trailing_stop = current_price * (1 - stop_loss_pct)
                trailing_stop = max(trailing_stop, new_trailing_stop)
            
            # Pozisyon kapatma koşulları
            if (current_price < trailing_stop or  # Trailing stop
                data['Combined_Signal'].iloc[i] == -1 or  # Satış sinyali
                (current_price - entry_price) / entry_price > take_profit_pct):  # Take profit
                
                data.loc[data.index[i], 'Position'] = 0
                position_open = False
    
    return data

In [8]:
# Performans analizi
def analyze_performance(data):
    data['Returns'] = data['Close'].pct_change()
    data['Strategy_Returns'] = data['Position'].shift(1) * data['Returns']
    
    data['Cumulative_Returns'] = (1 + data['Returns']).cumprod()
    data['Strategy_Cumulative_Returns'] = (1 + data['Strategy_Returns']).cumprod()
    
    # Performans metrikleri
    total_return = data['Strategy_Cumulative_Returns'].iloc[-1] - 1
    annual_return = (1 + total_return) ** (252/len(data)) - 1
    
    # Drawdown hesaplama
    rolling_max = data['Strategy_Cumulative_Returns'].cummax()
    drawdown = (data['Strategy_Cumulative_Returns'] - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
    
    print(f'Toplam Getiri: {total_return:.2%}')
    print(f'Yıllık Getiri: {annual_return:.2%}')
    print(f'Maksimum Drawdown: {max_drawdown:.2%}')
    
    return data

In [9]:

# Görselleştirme
def plot_strategy(data):
    fig = make_subplots(rows=4, cols=1, 
                        shared_xaxes=True,
                        vertical_spacing=0.05,
                        row_heights=[0.4, 0.2, 0.2, 0.2],
                        subplot_titles=('Fiyat ve Sinyaller', 'RSI', 'MACD', 'ADX'))

    # Ana grafik
    fig.add_trace(go.Candlestick(x=data.index,
                                open=data['Open'],
                                high=data['High'],
                                low=data['Low'],
                                close=data['Close'],
                                name='BTC-USD'), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=data.index, y=data['SMA20'], 
                            name='SMA20', line=dict(color='orange')), row=1, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['SMA50'], 
                            name='SMA50', line=dict(color='blue')), row=1, col=1)
    
    # Alım-satım sinyalleri
    buy_signals = data[data['Combined_Signal'] == 1].index
    sell_signals = data[data['Combined_Signal'] == -1].index
    
    fig.add_trace(go.Scatter(x=buy_signals, 
                            y=data.loc[buy_signals, 'Low'] * 0.99,
                            mode='markers',
                            marker=dict(symbol='triangle-up', size=15, color='green'),
                            name='Alış'), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=sell_signals, 
                            y=data.loc[sell_signals, 'High'] * 1.01,
                            mode='markers',
                            marker=dict(symbol='triangle-down', size=15, color='red'),
                            name='Satış'), row=1, col=1)
    
    # RSI
    fig.add_trace(go.Scatter(x=data.index, y=data['RSI'], 
                            name='RSI', line=dict(color='purple')), row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="red", row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", row=2, col=1)
    
    # MACD
    fig.add_trace(go.Scatter(x=data.index, y=data['MACD'], 
                            name='MACD', line=dict(color='blue')), row=3, col=1)
    fig.add_trace(go.Scatter(x=data.index, y=data['Signal_Line'], 
                            name='Signal Line', line=dict(color='orange')), row=3, col=1)
    fig.add_trace(go.Bar(x=data.index, y=data['MACD_Histogram'], 
                        name='Histogram'), row=3, col=1)
    
    # ADX
    fig.add_trace(go.Scatter(x=data.index, y=data['ADX'], 
                            name='ADX', line=dict(color='brown')), row=4, col=1)
    fig.add_hline(y=25, line_dash="dash", line_color="gray", row=4, col=1)
    
    # Grafik düzeni
    fig.update_layout(
        title='Gelişmiş Trend Takip Stratejisi',
        xaxis_title='Tarih',
        yaxis_title='Fiyat',
        template='plotly_white',
        height=1200,
        xaxis_rangeslider_visible=False
    )
    
    fig.show()

In [10]:
def run_strategy(symbol="BTC-USD", start="2022-01-01", end=None, 
                stop_loss_pct=0.03, take_profit_pct=0.06):
    # Veriyi indir
    data = download_data(symbol, start, end)
    
    # Göstergeleri hesapla
    data = calculate_indicators(data)
    data = calculate_adx(data)
    
    # Sinyalleri oluştur
    data = generate_signals(data)
    
    # Risk yönetimini uygula
    data = apply_risk_management(data, stop_loss_pct, take_profit_pct)
    
    # Performansı analiz et
    data = analyze_performance(data)
    
    # Görselleştir
    plot_strategy(data)
    
    return data

In [11]:
run_strategy()

[*********************100%%**********************]  1 of 1 completed
Toplam Getiri: -13.66%
Yıllık Getiri: -3.30%
Maksimum Drawdown: -17.48%


,Open,High,Low,Close,Adj Close,Volume,SMA20,SMA50,RSI,MACD,...,SMA_Signal,RSI_Signal,MACD_Signal,Strong_Trend,Combined_Signal,Position,Returns,Strategy_Returns,Cumulative_Returns,Strategy_Cumulative_Returns
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-01,46311.746094,47827.312500,46288.484375,47686.812500,47686.812500,24582667004,NaN,NaN,NaN,0.000000,...,0,0,0,False,0,0,NaN,NaN,NaN,NaN
2022-01-02,47680.925781,47881.406250,46856.937500,47345.218750,47345.218750,27951569547,NaN,NaN,NaN,-27.249644,...,0,0,-1,False,0,0,-0.007163,-0.0,0.992837,1.000000
2022-01-03,47343.542969,47510.726562,45835.964844,46458.117188,46458.117188,33071628362,NaN,NaN,NaN,-119.054479,...,0,0,-1,False,0,0,-0.018737,-0.0,0.974234,1.000000
2022-01-04,46458.851562,47406.546875,45752.464844,45897.574219,45897.574219,42494677905,NaN,NaN,NaN,-234.340277,...,0,0,-1,False,0,0,-0.012066,-0.0,0.962479,1.000000
2022-01-05,45899.359375,46929.046875,42798.222656,43569.003906,43569.003906,36851084859,NaN,NaN,NaN,-507.748260,...,0,0,-1,False,0,0,-0.050734,-0.0,0.913649,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-04,98106.992188,98734.429688,97562.976562,98236.226562,98236.226562,22342608078,97241.358594,97088.172656,52.363475,-150.206812,...,1,0,-1,True,0,0,0.001313,0.0,2.060029,0.863358
2025-01-05,98233.906250,98813.304688,97291.765625,98314.960938,98314.960938,20525254825,96855.620703,97243.302344,58.291410,15.609640,...,-1,0,1,True,0,0,0.000801,0.0,2.061680,0.863358
2025-01-06,98314.953125,102482.875000,97926.148438,102078.085938,102078.085938,51823432705,96652.494922,97487.947031,66.280193,445.537234,...,-1,0,1,True,0,0,0.038276,0.0,2.140594,0.863358
